# Question answering using vector store search
GPT excels at answering questions, but only on topics it remembers from its training data.
What should you do if you want GPT to answer questions about unfamiliar topics? E.g.,
- Recent events after Sep 2021
- Your non-public documents
- Information from past conversations
- etc.

This notebook demonstrates a two-step Search-Ask method for enabling GPT to answer questions using a library of reference text.

 1. **Search:** search your library of text for relevant text sections
 2. **Ask:** insert the retrieved text sections into a message to GPT and ask it the question"

## Why search is better than fine-tuning

GPT can learn knowledge in two ways:

 - Via model weights (i.e., fine-tune the model on a training set)
 - Via model inputs (i.e., insert the knowledge into an input message)

Although fine-tuning can feel like the more natural option—training on data is how GPT learned all of its other knowledge, after all—we generally do not recommend it as a way to teach the model knowledge. Fine-tuning is better suited to teaching specialized tasks or styles, and is less reliable for factual recall.

As an analogy, model weights are like long-term memory. When you fine-tune a model, it's like studying for an exam a week away. When the exam arrives, the model may forget details, or misremember facts it never read.

In contrast, message inputs are like short-term memory. When you insert knowledge into a message, it's like taking an exam with open notes. With notes in hand, the model is more likely to arrive at correct answers.

One downside of text search relative to fine-tuning is that each model is limited by a maximum amount of text it can read at once:

| Model           | Maximum text length       |
|-----------------|---------------------------|
| `gpt-3.5-turbo` | 4,096 tokens (~5 pages)   |
| `gpt-4`         | 8,192 tokens (~10 pages)  |
| `gpt-4-32k`     | 32,768 tokens (~40 pages) |

Continuing the analogy, you can think of the model like a student who can only look at a few pages of notes at a time, despite potentially having shelves of textbooks to draw upon.

Therefore, to build a system capable of drawing upon large quantities of text to answer questions, we recommend using a Search-Ask approach.
Continuing the analogy, you can think of the model like a student who can only look at a few pages of notes at a time, despite potentially having shelves of textbooks to draw upon.

Therefore, to build a system capable of drawing upon large quantities of text to answer questions, we recommend using a Search-Ask approach.

## Search
Text can be searched in many ways. E.g.,
- Lexical-based search
- Graph-based search
- Embedding-based search

This example notebook uses embedding-based search. [Embeddings](https://platform.openai.com/docs/guides/embeddings) are simple to implement and work especially well with questions, as questions often don't lexically overlap with their answers.

Consider embeddings-only search as a starting point for your own system. Better search systems might combine multiple search methods, along with features like popularity, recency, user history, redundancy with prior search results, click rate data, etc. Q&A retrieval performance may also be improved with techniques like [HyDE](https://arxiv.org/abs/2212.10496), in which questions are first transformed into hypothetical answers before being embedded. Similarly, GPT can also potentially improve search results by automatically transforming questions into sets of keywords or search terms.

## Full procedure
Specifically, this notebook demonstrates the following procedure:
1. Prepare search data (once per document)
    1. Collect: We'll download a few hundred Wikipedia articles about the 2022 Olympics
    2. Chunk: Documents are split into short, mostly self-contained sections to be embedded
    3. Embed: Each section is embedded with the OpenAI API
    4. Store: Embeddings are saved (for large datasets, use a vector database)
2. Search (once per query)
    1. Given a user question, generate an embedding for the query from the OpenAI API
    2. Using the embeddings, rank the text sections by relevance to the query
3. Ask (once per query)
    1. Insert the question and the most relevant sections into a message to GPT
    2. Return GPT's answer

### Costs
Because GPT is more expensive than embeddings search, a system with a decent volume of queries will have its costs dominated by step 3.

- For `gpt-3.5-turbo` using ~1,000 tokens per query, it costs ~$0.002 per query, or ~500 queries per dollar (as of Apr 2023)
- For `gpt-4`, again assuming ~1,000 tokens per query, it costs ~$0.03 per query, or ~30 queries per dollar (as of Apr 2023)
Of course, exact costs will depend on the system specifics and usage patterns.

## Preamble
We'll begin by:
- Importing the necessary libraries
- Selecting models for embeddings search and question answering

## Installation
Install the Azure Open AI SDK using the below command.

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.9"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.9

In [2]:
#r "nuget:Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23552.1"

using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AIUtilities;

Installed Packages Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23552.1

## Run this cell, it will prompt you for the apiKey, endPoint, embeddingDeployment, and chatDeployment

In [3]:
var azureOpenAIKey = await Kernel.GetPasswordAsync("Provide your OPEN_AI_KEY");

// Your endpoint should look like the following https://YOUR_OPEN_AI_RESOURCE_NAME.openai.azure.com/
var azureOpenAIEndpoint = await Kernel.GetInputAsync("Provide the OPEN_AI_ENDPOINT");

// Enter the deployment name you chose when you deployed the model.
var embeddingDeployment = await Kernel.GetInputAsync("Provide embedding deployment name");
var chatDeployment = await Kernel.GetInputAsync("Provide chat deployment name");

### Import namesapaces and create an instance of `OpenAiClient` using the `azureOpenAIEndpoint` and the `azureOpenAIKey`

In [4]:
using Azure;
using Azure.AI.OpenAI;

In [5]:
OpenAIClient client = new (new Uri(azureOpenAIEndpoint), new AzureKeyCredential(azureOpenAIKey.GetClearTextPassword()));

## Load embedding data

**IMPORTANT** In this sample, we'll be loading *wikipedia_embeddings.json*. This file is generated by running the [Embedding_Wikipedia_articles_for_search.ipynb](../../../Embeddings/dotnet/csharp/Embedding_Wikipedia_articles_for_search.ipynb) notebook.

In [6]:
public record PageBlockWithEmbeddings(string PageTitle, string Block, float[] Embedding);

In [7]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;

var filePath = Path.Combine("..","..","..","Data","wikipedia_embeddings.json");

var olympicsData = JsonSerializer.Deserialize<PageBlockWithEmbeddings[]>(File.ReadAllText(filePath));

In [8]:
olympicsData.Take(4).DisplayTable();

PageTitle,Block,Embedding
2022 Winter Olympics,"The 2022 Winter Olympics, officially called the XXIV Olympic Winter Games () and commonly known as Beijing 2022 (2022), was an international winter multi-sport event held from 4 to 20 February 2022 in Beijing, China, and surrounding areas with competition in selected events beginning 2 February 2022.{{cite web|title=SuperSport|url=https://supersport.com/news/cd6663a2-8236-44d8-a8b9-4fa192190da7/%7B%7B%20url()-%3Ecurrent()%20%7D%7D|access-date=25 February 2022|website=supersport.com|language=ZA|archive-date=25 February 2022|archive-url=https://web.archive.org/web/20220225173447/https://supersport.com/news/cd6663a2-8236-44d8-a8b9-4fa192190da7/%7B%7B%20url()-%3Ecurrent()%20%7D%7D|url-status=live}} It was the 24th edition of the Winter Olympic Games.","[ -0.007000031, -0.025182178, -0.010695381, -0.005645496, -0.026770474, 0.010612124, -0.01401287, -0.00023596284, -0.0060201543, -0.022453895, 0.03729934, 0.0129561415, -0.030408185, -0.023824442, -0.014550841, -0.027718328, 0.016702726, -0.0068463245, 0.0015754872, -0.015831726 ... (1516 more) ]"
2022 Winter Olympics,"Beijing was selected as host city on 31 July 2015 at the 128th IOC Session in Kuala Lumpur, Malaysia, marking its second time hosting the Olympics, and the last of three consecutive Olympics hosted in East Asia following the 2018 Winter Olympics in Pyeongchang County, South Korea, and the 2020 Summer Olympics in Tokyo, Japan. Having previously hosted the 2008 Summer Olympics, Beijing became the first city to have hosted both the Summer and Winter Olympics. The venues for the Games were concentrated around Beijing, its suburb Yanqing District, and Zhangjiakou, with some events (including the ceremonies and curling) repurposing venues originally built for Beijing 2008 (such as Beijing National Stadium and the Beijing National Aquatics Centre).","[ 0.008893254, -0.012699212, -0.007250349, -0.0007829965, -0.020767841, 0.024421562, -0.03750137, -0.0052427067, -0.0037869278, -0.019334264, 0.016835019, 0.010022355, -0.0055186385, -0.0069966186, -0.02300067, -0.008538032, 0.014373833, -0.011024591, -0.0111577995, 0.0054932656 ... (1516 more) ]"
2022 Winter Olympics,"The Games featured a record 109 events across 15 disciplines, with big air freestyle skiing and women's monobob making their Olympic debuts as medal events, as well as several new mixed competitions. A total of 2,871 athletes representing 91 teams competed in the Games, with Haiti and Saudi Arabia making their Winter Olympic debut.","[ -0.009414442, 0.0101670865, -0.0019868554, -0.023944318, -0.0073287217, 0.01984942, -0.017017433, 0.0011855755, -0.00017719848, -0.03722404, 0.017514944, 0.006266727, -0.013177667, -0.025602689, -0.0065027256, -0.001007779, 0.029697588, -0.017132243, -0.01583106, -0.0075710993 ... (1516 more) ]"
2022 Winter Olympics,"Beijing's hosting of the Games was subject to various concerns and controversies including those related to human rights violations in China, such as the Uyghur genocide, which led to calls for a boycott of the games.{{Cite news|last=Reyes|first=Yacob|date=8 December 2021|title=Beijing Olympics: These countries have announced diplomatic boycotts|work=[[Axios (website)|Axios]]|url=https://www.axios.com/diplomatic-boycott-beijing-olympics-list-countries-73e1240f-b925-40bf-ae67-648e774971c8.html|access-date=5 February 2022|archive-date=4 February 2022|archive-url=https://web.archive.org/web/20220204210817/https://www.axios.com/diplomatic-boycott-beijing-olympics-list-countries-73e1240f-b925-40bf-ae67-648e774971c8.html|url-status=live}}{{Cite news|last1=Allen-Ebrahimian|first1=Bethany|last2=Baker|first2=Kendall|date=1 February 2022|title=The IOC stays silent on human rights in China|work=[[Axios (website)|Axios]]|url=https://www.axios.com/winter-olympics-beijing-ioc-silence-human-rights-31ec1273-d894-4a67-993b-4b4156d42d44.html|access-date=5 February 2022|archive-date=5 February 2022|archive-url=https://web.archive.org/web/202

## 2. Define schema

In [9]:
#r "nuget: Milvus.Client, 2.2.2-preview.6"

Installed Packages Milvus.Client, 2.2.2-preview.6

In [10]:
using Milvus.Client;

In [11]:
var milvusClient = new MilvusClient("40.88.221.217");

In [12]:
await milvusClient.GetCollection("wikipediawinterolympics2022").DropAsync()

Error: Command cancelled.

### Define schema

In [38]:
var schema = new CollectionSchema
{
    Fields =
    {
        FieldSchema.Create<long>("block_id", isPrimaryKey: true),
        FieldSchema.CreateVarchar("page_name", maxLength: 300),
        FieldSchema.CreateVarchar("content_block", maxLength: 5000),
        FieldSchema.CreateFloatVector("content_embedding", dimension: 1536)
    },
    Description = "Wikipedia 2022 Winter Olympics Collection",
    EnableDynamicFields = true
};

### Create collection 

In [39]:
var collection = await milvusClient.CreateCollectionAsync(collectionName: "wikipediawinterolympics2022",schema: schema, shardsNum: 2);

In [40]:
var dataCollectionWithId = 
    olympicsData
        .Select((x,i) => new {
            Id=(long)i,
            PageTitle = x.PageTitle, 
            Block = x.Block, 
            Embedding = x.Embedding
            });

In [41]:
var blockIds = dataCollectionWithId.Select(x => x.Id).ToArray();
var pageTitles = dataCollectionWithId.Select(x => x.PageTitle).ToArray();
var contentBlocks = dataCollectionWithId.Select(x => x.Block).ToArray();
var contentEmbeddings = dataCollectionWithId.Select(x => new ReadOnlyMemory<float>(x.Embedding)).ToArray();

In [42]:
await collection.InsertAsync(new FieldData[]
{
    FieldData.Create("block_id",blockIds),
    FieldData.Create("page_name", pageTitles),
    FieldData.Create("content_block",contentBlocks),
    FieldData.CreateFloatVector("content_embedding",contentEmbeddings)
})

Milvus.Client.MutationResult Acknowledged False Timestamp 445074587497267205 InsertCount 389 DeleteCount 0 UpsertCount 0 SuccessIndex [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19 ... (369 more) ] ErrorIndex [ ] Ids Milvus.Client.MilvusIds LongIds [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19 ... (369 more) ] StringIds <null>

### Persist data

In [43]:
await collection.FlushAsync();

 ## 2. Search
    
 
 Now we'll define a search function that:
 - Takes a user query and a dataframe with text & embedding columns
 - Embeds the user query with the OpenAI API
 - Uses distance between query embedding and text embeddings to rank the texts
 - Returns two lists:
    - The top N texts, ranked by relevance
    - Their corresponding relevance scores

### Create index

In [44]:
await collection.CreateIndexAsync(
    fieldName: "content_embedding", 
    indexType: IndexType.Flat, 
    metricType: SimilarityMetricType.L2, 
    extraParams: new Dictionary<string,string> {["nlist"] = "1024"}, 
    indexName: "pageblock_idx");

### Load collection

In [45]:
await collection.LoadAsync();
await collection.WaitForCollectionLoadAsync();

### Define search function

- Generates embeddings for user query
- Builds query parameters for Milvus search client to extract the page_name and content_block fields
- Run search
- Transform search results into an `IEnumerable<SearchResult>`

In [90]:
public record SearchResult(string PageTitle, string Text, float Score);

public async Task<IEnumerable<SearchResult>> SearchAsync(string query, int resultCount = 5){
    var response = await client.GetEmbeddingsAsync(embeddingDeployment, new EmbeddingsOptions(query));
    var queryEmbedding = response.Value.Data[0].Embedding.ToArray();

    var parameters = new SearchParameters
    {
        OutputFields = { "page_name","content_block" },
        ConsistencyLevel = ConsistencyLevel.Strong,
        ExtraParameters = { ["nprobe"] = "1024" }
    };

    var searchResults = await collection.SearchAsync(
        vectorFieldName: "content_embedding",
        vectors: new ReadOnlyMemory<float>[] { new ReadOnlyMemory<float>(queryEmbedding) },
        SimilarityMetricType.L2,
        limit: resultCount,
        parameters);

    var searchResultDict = searchResults.FieldsData.ToDictionary(x => x.FieldName, x => ((FieldData<string>)x).Data);

    return searchResults.Scores.Select((score,i) => 
        new SearchResult(
            searchResultDict["page_name"][i],
            searchResultDict["content_block"][i],
            score));
}

### Use `SearchAsync` to search the data

In [92]:
var search = await SearchAsync("curling gold medal", 3);

search.DisplayTable();

PageTitle,Text,Score
2022 Winter Olympics medal table,Two bronze medals were awarded to Daniela Maier and Fanny Smith for a third-place tie in the freestyle women's ski cross event following a decision by the Court of Arbitration for Sport.{{cite web|url=https://www.tas-cas.org/fileadmin/user_upload/CAS_Media_Release_8741.pdf|title=Court of Arbitration for Sport Media Release|access-date=13 December 2022|publisher=[[Court of Arbitration for Sport]]|date=13 December 2022|archive-date=13 December 2022|archive-url=https://web.archive.org/web/20221213171856/https://www.tas-cas.org/fileadmin/user_upload/CAS_Media_Release_8741.pdf|url-status=live}},0.31844968
2022 Winter Olympics medal table,"Biathletes Johannes Thingnes Bø, Quentin Fillon Maillet, and Marte Olsbu Røiseland, and cross-country skier Alexander Bolshunov won the most total medals at the games with five each.{{cite web |title=Beijing 2022 |url=https://www.teamgb.com/competitions/beijing-2022/6dWdXrzU85Vn1jF6ZC9Onl |publisher=[[British Olympic Association]] |access-date=26 February 2022 |archive-date=18 March 2022 |archive-url=https://web.archive.org/web/20220318145104/https://www.teamgb.com/competitions/beijing-2022/6dWdXrzU85Vn1jF6ZC9Onl |url-status=live }} Bø also earned the most gold medals with four.{{cite news |author=[[Agence France-Presse]] |title=Norwegian Biathlete Boe Gets Fourth Beijing Olympics Gold Medal |url=https://www.barrons.com/news/norwegian-biathlete-boe-gets-fourth-beijing-olympics-gold-medal-01645189808 |access-date=27 March 2022 |work=[[Barron's (newspaper)|Barron's]] |date=18 February 2022 |archive-date=22 February 2023 |archive-url=https://web.archive.org/web/20230222200651/https://www.barrons.com/news/norwegian-biathlete-boe-gets-fourth-beijing-olympics-gold-medal-01645189808 |url-status=live }} Snowboarder Zoi Sadowski-Synnott of New Zealand won the first Winter Olympic gold medal for that nation.{{cite news |first1=Bryan Armen |last1=Graham |title=Zoi Sadowski-Synnott Wins New Zealand's First Ever Winter Olympic Gold |url=https://www.theguardian.com/sport/2022/feb/05/zoi-sadowski-synnott-new-zealand-first-winter-olympic-gold-snowboard-beijing-2022-tess-coady |access-date=12 July 2022 |work=[[The Guardian]] |date=5 February 2022 |archive-date=26 February 2022 |archive-url=https://web.archive.org/web/20220226175310/https://www.theguardian.com/sport/2022/feb/05/zoi-sadowski-synnott-new-zealand-first-winter-olympic-gold-snowboard-beijing-2022-tess-coady |url-status=live }} Germany achieved a podium sweep in the men's two-man bobsleigh competition with Francesco Friedrich and Thorsten Margis winning gold, Johannes Lochner and Florian Bauer earning silver, and Christoph Hafer and Matthias Sommer attaining bronze.{{cite news |last1=Levinsohn |first1=Dan |title=Germany Sweeps Two-Man Bobsled Podium with Friedrich, Lochner, Hafer |url=https://www.nbcolympics.com/news/recap-two-man-final-heats |access-date=19 February 2022 |agency=[[NBC Sports]] |date=15 February 2022 |archive-date=19 March 2023 |archive-url=https://web.archive.org/web/20230319113128/https://www.nbcolympics.com/news/recap-two-man-final-heats |url-status=live }}",0.32276392
2022 Winter Olympics,"thumb|Medals of 2022 Winter Olympics Norway finished at the top of the medal table for the second successive Winter Olympics, winning a total of 37 medals, of which 16 were gold, setting a new record for the largest number of gold medals won at a single Winter Olympics. Germany finished second with 12 golds and 27 medals overall, and the host nation China finished third with nine gold medals, marking their most successful performance in Winter Olympics history. The team representing the ROC ended up with the second largest number of medals won at the Games, with 32, but finished ninth on the medal table, as only six gold medals were won by the delegation. Traditional Winter powerhouse Canada; despite having won 26 medals, only four of them were gold, resulting in a finish outside the top ten in 

## 3.Ask
With the search function above, we can now automatically retrieve relevant knowledge and insert it into messages to GPT.

Below, we define a function `AskAsync` that:

 - Takes a user query
 - Searches for text relevant to the query
 - Stuffs that text into a message for GPT
 - Sends the message to GPT
 - Returns GPT's answer

In [99]:
var tokenizer = await Tokenizer.CreateAsync(TokenizerModel.gpt35);

public async Task<string> AskAsync(string question){

    var searchResults = await SearchAsync(question, 3);

    var articles = string.Join("\n", searchResults.Select(s => $"""
    Wikipedia article section:
    {s.Text}

    """));

    var userQuestion = $"""""
                Use the below articles on the 2022 Winter Olympics to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."
                
                {articles}
                
                Question: {question}
                """"";

    var options= new ChatCompletionsOptions{
        Messages =
            {
                new ChatMessage(ChatRole.System, @"You answer questions about the 2022 Winter Olympics."),
                new ChatMessage(ChatRole.User, userQuestion)
            },
        Temperature = 0f,
        MaxTokens = 3500,
        DeploymentName = chatDeployment
    };

    var response = await client.GetChatCompletionsAsync(options);

    var answer = response.Value.Choices.FirstOrDefault()?.Message?.Content;  
    return answer;
}

In [100]:
await AskAsync("How many gold medals in total?")

In total, there were 16 gold medals won at the 2022 Winter Olympics.

In [101]:
await AskAsync("Where did the 2022 winter Olympics took place?")

The 2022 Winter Olympics took place in Beijing, China.